# 第 11 章: 運用管理 - モニタリング


本ノートブックでは、「メトリクス出力先をカスタマイズする」節で紹介されている、MetricsReporter インターフェースを実装した `IcebergTableMetricsReporter` を利用できます。この Custom Metrics Reporter は、各テーブルに対するクエリ・書き込みに対して出力されるメトリクスを Iceberg テーブルに書き込みます。本ノートブックを実行するには、最初に以下の「事前準備」を完了させてください。

なお、各種設定内容の詳細については、[Example of Custom Metrics Reporter](https://github.com/murashitas/iceberg_book_handson/tree/main/custom-metrics-reporter-iceberg) を参照してください。

## 事前準備

1. [Example of Custom Metrics Reporter](https://github.com/murashitas/iceberg_book_handson/tree/main/custom-metrics-reporter-iceberg) の内容を参考に JAR ファイルをビルドする
2. ビルドが完了したら JAR ファイルを、本ノートブックから `/home/jovyan/jar/` 配下にアップロードする
    1. ノートブックの左側のペインから、`jar` ディレクトリを選択します
    2. 上の ↑ のアイコンをクリックし、作成した JAR ファイルをアップロードします
3. アップロードが完了したら各セルを実行します。

## SparkSession を初期化する
`metrics-catalog-name`に対して設定されている`METRICS_CATALOG` は、Iceberg テーブルに対してメトリクスを出力する際に利用されるカタログです。メトリクスの出力先テーブルに対しては、様々なカタログを選択することができますが、以下の例では、既存の REST カタログを利用しています。

In [ ]:
import time
import pyspark
from pyspark.sql import SparkSession
CATALOG = "my_catalog"
METRICS_CATALOG = "metrics_catalog"
CATALOG_URL = "http://server:8181/"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.8.1"

In [ ]:
spark = (
    SparkSession.builder
        .config("spark.jars", "/home/jovyan/jar/custom-metrics-reporter-iceberg-0.1.jar")
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint",S3_ENDPOINT)
        .config(f"spark.sql.catalog.{CATALOG}.view-endpoints-supported", "true")
        .config(f"spark.sql.catalog.{CATALOG}.metrics-reporter-impl", "jp.gihyo.iceberg.IcebergTableMetricsReporter")
        .config(f"spark.sql.catalog.{CATALOG}.metrics-catalog-name", METRICS_CATALOG)
        .config(f"spark.sql.catalog.{METRICS_CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{METRICS_CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{METRICS_CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{METRICS_CATALOG}.s3.endpoint",S3_ENDPOINT)
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", "my_catalog")
        .getOrCreate()
)

In [ ]:
%sql spark

### (Optional) データベースの作成
データベースを作成していない場合、以下のセルを実行してください。既にデータベースが存在する場合は、本ステップにつきましてはスキップしてください。

In [ ]:
%%sql
CREATE DATABASE IF NOT EXISTS db

##　各種メトリクス収集用のテーブルを作成する

In [ ]:
%%sql
CREATE OR REPLACE TABLE db.sales_iceberg_m (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string) 
USING iceberg

メトリクス収集用テーブルに対して 5 レコードを追加する。このタイミングで `ScanReport` (読み込み)と `CommitReport` (書き込み)を収集する 2 つの Iceberg テーブルが作成されます。また、書き込み 5 レコードに関する `CommitReport` が Commit metrics 用の Iceberg テーブル `metrics_report_commit` に書き込まれます。

In [ ]:
%%sql
INSERT INTO db.sales_iceberg_m VALUES
    ('tomato juice', 2.00, 1698, 'DRE8DLTFNX0MLCE8DLTFNX0MLC', TIMESTAMP '2023-07-18T02:20:58Z', 'drink'),
    ('cocoa', 2.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2024-08-26T11:36:48Z', 'drink'),
    ('espresso', 2.00, 1037, 'DRBFZUJWPZ9SRABFZUJWPZ9SRA', TIMESTAMP '2024-04-19T12:17:22Z', 'drink'),
    ('broccoli', 1.00, 3092, 'GRK0L8ZQK0L8ZQ', TIMESTAMP '2023-03-22T18:48:04Z', 'grocery'),
    ('nutmeg', 1.00, 3512, 'GR15U0LKA15U0LKA', TIMESTAMP '2024-02-27T15:13:31Z', 'grocery')

`ScanReport` および `CommitReport` 用のテーブルが作成されたか確認してみましょう。デフォルトの設定の場合 `report_commit` と `report_scan` テーブルが作成されることを確認できます。

In [ ]:
%%sql
SHOW TABLES IN db

## `CommitReport` の確認
`report_commit` テーブルにクエリすることで、書き込みに関するメトリクスを収集できます。

In [ ]:
%%sql
SELECT * FROM metrics_catalog.db.report_commit

## `ScanReport` の確認
次に、`ScanReport` についても確認してみましょう。`sales_iceberg_m` に対してクエリを行うことで、本メトリクスが出力されます。

In [ ]:
%%sql
SELECT category, count(*) as cat_cnt  FROM db.sales_iceberg_m GROUP BY category

`ScanReport` の出力先テーブルである `report_scan` に対してクエリを実行することで、先ほど出力された `ScanReport` メトリクスを確認できます。

In [ ]:
%%sql
SELECT * FROM metrics_catalog.db.report_scan